In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Does nn.Conv2d init work well?

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [77]:
torch.nn.modules.conv._ConvNd.reset_parameters??

    def reset_parameters(self):
        n = self.in_channels
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)


In [65]:
init.kaiming_normal_??

Fills the input `Tensor` with values according to the method
described in "Delving deep into rectifiers: Surpassing human-level
performance on ImageNet classification" - He, K. et al. (2015), using a
normal distribution. The resulting tensor will have values sampled from
$\mathcal{N}(0, \text{std})$ where

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan\_in}}}$$

Also known as He initialization.

In [66]:
init.kaiming_uniform_??

Fills the input `Tensor` with values according to the method
described in "Delving deep into rectifiers: Surpassing human-level
performance on ImageNet classification" - He, K. et al. (2015), using a
uniform distribution. The resulting tensor will have values sampled from
$\mathcal{U}(-\text{bound}, \text{bound})$ where

$$\text{bound} = \sqrt{\frac{6}{(1 + a^2) \times \text{fan\_in}}}$$

6 here is 2x3; 2 comes from 

    def gain(a): return math.sqrt(2.0 / (1 + a**2)); 

3 comes from math.sqrt(3.), std of uniform(-1,1)

In [7]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [8]:
x_train.shape,x_valid.shape

(torch.Size([50000, 784]), torch.Size([10000, 784]))

In [9]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [10]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [27]:
l1 = nn.Conv2d(1, nh, 5)

In [22]:
x = x_valid[:100]

In [23]:
x.shape

torch.Size([100, 1, 28, 28])

In [24]:
def stats(x): return x.mean(),x.std()

In [25]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [28]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0073, grad_fn=<MeanBackward1>),
  tensor(0.1147, grad_fn=<StdBackward0>)),
 (tensor(-0.0464, grad_fn=<MeanBackward1>),
  tensor(0.1051, grad_fn=<StdBackward0>)))

In [29]:
t = l1(x)

In [30]:
t.shape

torch.Size([100, 32, 24, 24])

In [34]:
stats(t)

(tensor(-0.0282, grad_fn=<MeanBackward1>),
 tensor(0.6568, grad_fn=<StdBackward0>))

In [35]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0674, grad_fn=<MeanBackward1>),
 tensor(1.1172, grad_fn=<StdBackward0>))

In [36]:
import torch.nn.functional as F

In [78]:
def f1(x,a=0): return F.leaky_relu(l1(x),a) # a=0 -> regular relu

In [38]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5479, grad_fn=<MeanBackward1>),
 tensor(0.9823, grad_fn=<StdBackward0>))

In [45]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2067, grad_fn=<MeanBackward1>),
 tensor(0.3614, grad_fn=<StdBackward0>))

In [46]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [47]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [51]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [52]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [53]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [54]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [55]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [56]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5764)

In [57]:
1/math.sqrt(3.)

0.5773502691896258

In [58]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

the above is the same as `init.kaiming_normal_`:

    The resulting tensor will have values sampled from
$\mathcal{U}(-\text{bound}, \text{bound})$ where

$$\text{bound} = \sqrt{\frac{6}{(1 + a^2) \times \text{fan\_in}}}$$

seems it should be this - for relu, leak a=0

In [59]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5666, grad_fn=<MeanBackward1>),
 tensor(1.0632, grad_fn=<StdBackward0>))

but why does pytorch use this? how can leak a=sqrt(5)?

In [60]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2067, grad_fn=<MeanBackward1>),
 tensor(0.3576, grad_fn=<StdBackward0>))

In [61]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [67]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [68]:
y = y_valid[:100].float()

In [69]:
t = m(x)
stats(t)

(tensor(0.0311, grad_fn=<MeanBackward1>),
 tensor(0.0070, grad_fn=<StdBackward0>))

In [70]:
l = mse(t,y)
l.backward()

In [71]:
stats(m[0].weight.grad)

(tensor(-0.0040), tensor(0.0275))

In [72]:
init.kaiming_uniform_??

In [73]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [74]:
t = m(x)
stats(t)

(tensor(0.2137, grad_fn=<MeanBackward1>),
 tensor(0.2321, grad_fn=<StdBackward0>))

In [75]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(0.0728), tensor(0.2610))

## Export

In [76]:
!./notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py
